In [1]:
# MLB Stats API Python wrapper
import statsapi

In [2]:
# Get MLB sports data
MLB_CODE = "mlb"
sports = statsapi.get("sports", {}).get("sports")
mlb_sport = next((sport for sport in sports if sport.get("code") == MLB_CODE), None)
sports_data = [(sport["id"], sport["name"], sport["code"]) for sport in sports]
print(sports_data)
mlb_id = mlb_sport.get("id")

# Important Meta Data
pitch_types = statsapi.get("meta", {"type": "pitchTypes"})
pitch_codes = statsapi.get("meta", {"type": "pitchCodes"})

[(1, 'Major League Baseball', 'mlb'), (11, 'Triple-A', 'aaa'), (12, 'Double-A', 'aax'), (13, 'High-A', 'afa'), (14, 'Single-A', 'afx'), (16, 'Rookie', 'rok'), (17, 'Winter Leagues', 'win'), (21, 'Minor League Baseball', 'min'), (23, 'Independent Leagues', 'ind'), (61, 'Negro League Baseball', 'nlb'), (32, 'Korean Baseball Organization', 'kor'), (31, 'Nippon Professional Baseball', 'jml'), (51, 'International Baseball', 'int'), (509, 'International Baseball (18U)', 'nae'), (510, 'International Baseball (16 and under)', 'nas'), (6005, 'International Baseball (amateur)', 'ame'), (22, 'College Baseball', 'bbc'), (586, 'High School Baseball', 'hsb')]


In [3]:
aaa_players = statsapi.get("sports_players", {"sportId": 11, "season": 2024}).get("people")
mexican_players = [player for player in aaa_players if player["birthCountry"].lower() == "mexico"]
print(len(mexican_players))
mexican_position_players = [player for player in mexican_players if player["primaryPosition"]["code"] != "1"]
print(len(mexican_position_players))
print(mexican_players[0])

31
13
{'id': 666018, 'fullName': 'Jonathan Aranda', 'link': '/api/v1/people/666018', 'firstName': 'Jonathan', 'lastName': 'Aranda', 'primaryNumber': '62', 'birthDate': '1998-05-23', 'currentAge': 26, 'birthCity': 'Tijuana', 'birthStateProvince': 'Baja California', 'birthCountry': 'Mexico', 'height': '6\' 0"', 'weight': 210, 'active': True, 'currentTeam': {'id': 234, 'name': 'Durham Bulls', 'link': '/api/v1/teams/234'}, 'primaryPosition': {'code': '3', 'name': 'First Base', 'type': 'Infielder', 'abbreviation': '1B'}, 'useName': 'Jonathan', 'useLastName': 'Aranda', 'middleName': 'Alexander', 'boxscoreName': 'Aranda', 'gender': 'M', 'nameMatrilineal': 'Ventura', 'isPlayer': True, 'isVerified': False, 'mlbDebutDate': '2022-06-24', 'batSide': {'code': 'L', 'description': 'Left'}, 'pitchHand': {'code': 'R', 'description': 'Right'}, 'nameFirstLast': 'Jonathan Aranda', 'nameSlug': 'jonathan-aranda-666018', 'firstLastName': 'Jonathan Aranda', 'lastFirstName': 'Aranda, Jonathan', 'lastInitName':

In [6]:
# Retrieve list of MLB players for a given season
# THIS WILL BE IMPORTANT TO BUILD A PLAYER DATABASE THAT WE CAN USE TO UNIQUELY IDENTIFY PLAYERS WHEN BUILDING FEATURE VECTORS
# To get a sample of the player objects, inspect the first item in the list
# player = mlb_players[0]
# Next steps: build player database
# Important attributes: name, active, last_team, mlb_debut, primary_position, number, bat_side
PITCH_POS_CODE = "1"
mlb_players = statsapi.get("sports_players", {"sportId": mlb_id, "season": 2000}).get("people")
mlb_pitchers = [player for player in mlb_players if player.get("primaryPosition", {}).get("code") == PITCH_POS_CODE]
derek_jeter = next((player for player in mlb_players if player.get("fullName") == "Derek Jeter"), None)
roger_clemens = next((player for player in mlb_players if player.get("fullName") == "Roger Clemens"), None)
print(roger_clemens)

{'id': 112388, 'fullName': 'Roger Clemens', 'link': '/api/v1/people/112388', 'firstName': 'William', 'lastName': 'Clemens', 'primaryNumber': '22', 'birthDate': '1962-08-04', 'currentAge': 62, 'birthCity': 'Dayton', 'birthStateProvince': 'OH', 'birthCountry': 'USA', 'height': '6\' 4"', 'weight': 235, 'active': False, 'currentTeam': {'id': 147, 'name': 'New York Yankees', 'link': '/api/v1/teams/147'}, 'primaryPosition': {'code': '1', 'name': 'Pitcher', 'type': 'Pitcher', 'abbreviation': 'P'}, 'useName': 'Roger', 'useLastName': 'Clemens', 'middleName': 'Roger', 'boxscoreName': 'Clemens', 'nickName': 'Rocket', 'gender': 'M', 'isPlayer': True, 'isVerified': True, 'draftYear': 1983, 'lastPlayedDate': '2007-09-16', 'mlbDebutDate': '1984-05-15', 'batSide': {'code': 'R', 'description': 'Right'}, 'pitchHand': {'code': 'R', 'description': 'Right'}, 'nameFirstLast': 'Roger Clemens', 'nameSlug': 'roger-clemens-112388', 'firstLastName': 'Roger Clemens', 'lastFirstName': 'Clemens, Roger', 'lastInitNa

In [5]:
# Game Play by Play Data
# Game id is required; gather a list of games to retrieve each id for iteration
# Let's get a look at the pbp available data
games_20150701 = statsapi.schedule(sportId=11, start_date="01/01/2024", end_date="12/31/2024")
first_game = games_20150701[0]
print(first_game)
first_game_id = games_20150701[0].get("game_id")

pbp_data = statsapi.get("game_playByPlay", {"gamePk": first_game_id})
# For a pitch by pitch breakdown, look for plays where result => type == "atBat".
# Then, go for "playEvents" for an ordered list of at bat pitches.
# The line belows shows an example of a last pitch of the at bat.
pbp_data["allPlays"][7]
# pbp_data["allPlays"][1]["playEvents"][-1]

{'game_id': 764388, 'game_datetime': '2024-03-25T00:07:00Z', 'game_date': '2024-03-24', 'game_type': 'E', 'status': 'Final', 'away_name': 'Sacramento River Cats', 'home_name': 'San Francisco Giants', 'away_id': 105, 'home_id': 137, 'doubleheader': 'N', 'game_num': 1, 'home_probable_pitcher': '', 'away_probable_pitcher': 'Mason Black', 'home_pitcher_note': '', 'away_pitcher_note': '', 'away_score': 8, 'home_score': 1, 'current_inning': 9, 'inning_state': 'Bottom', 'venue_id': 2529, 'venue_name': 'Sutter Health Park', 'national_broadcasts': [], 'series_status': None, 'winning_team': 'Sacramento River Cats', 'losing_team': 'San Francisco Giants', 'winning_pitcher': 'Mason Black', 'losing_pitcher': 'Carson Seymour', 'save_pitcher': None, 'summary': '2024-03-24 - Sacramento River Cats (8) @ San Francisco Giants (1) (Final)'}


{'result': {'type': 'atBat',
  'event': 'Walk',
  'eventType': 'walk',
  'description': 'Austin Slater walks.',
  'rbi': 0,
  'awayScore': 2,
  'homeScore': 0,
  'isOut': False},
 'about': {'atBatIndex': 7,
  'halfInning': 'bottom',
  'isTopInning': False,
  'inning': 1,
  'startTime': '2024-03-25T00:26:14.158Z',
  'endTime': '2024-03-25T00:27:57.481Z',
  'isComplete': True,
  'isScoringPlay': False,
  'hasReview': False,
  'hasOut': False,
  'captivatingIndex': 0},
 'count': {'balls': 4, 'strikes': 2, 'outs': 1},
 'matchup': {'batter': {'id': 596103,
   'fullName': 'Austin Slater',
   'link': '/api/v1/people/596103'},
  'batSide': {'code': 'R', 'description': 'Right'},
  'pitcher': {'id': 696131,
   'fullName': 'Mason Black',
   'link': '/api/v1/people/696131'},
  'pitchHand': {'code': 'R', 'description': 'Right'},
  'postOnFirst': {'id': 596103,
   'fullName': 'Austin Slater',
   'link': '/api/v1/people/596103'},
  'batterHotColdZones': [],
  'pitcherHotColdZones': [],
  'splits': {'

In [7]:
# Get teams data
teams_response = statsapi.get("teams", {"sportId": mlb_id, "season": 2000})
print(teams_response)

{'copyright': 'Copyright 2024 MLB Advanced Media, L.P.  Use of any content on this page acknowledges agreement to the terms posted here http://gdx.mlb.com/components/copyright.txt', 'teams': [{'springLeague': {'id': 114, 'name': 'Cactus League', 'link': '/api/v1/league/114', 'abbreviation': 'CL'}, 'allStarStatus': 'N', 'id': 133, 'name': 'Oakland Athletics', 'link': '/api/v1/teams/133', 'season': 2000, 'venue': {'id': 10, 'name': 'Network Associates Coliseum', 'link': '/api/v1/venues/10'}, 'teamCode': 'oak', 'fileCode': 'oak', 'abbreviation': 'OAK', 'teamName': 'Athletics', 'locationName': 'Oakland', 'firstYearOfPlay': '1901', 'league': {'id': 103, 'name': 'American League', 'link': '/api/v1/league/103'}, 'division': {'id': 200, 'name': 'American League West', 'link': '/api/v1/divisions/200'}, 'sport': {'id': 1, 'link': '/api/v1/sports/1', 'name': 'Major League Baseball'}, 'shortName': 'Oakland', 'franchiseName': 'Oakland', 'clubName': 'Athletics', 'active': True}, {'springLeague': {'i